In [1]:
import python_utils.python_utils.caching as caching
import ml_stuff_run.ml_stuff_run.constants as constants
caching.init(constants.cache_folder, constants.which_hash)
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mpl_colors
import matplotlib.cm as cm
import pdb
import numpy as np
import scipy
import ml_stuff.ml_stuff.hier_variational_regr as hvr
#import causal_frl.causal_frl.hier_variational_regr as hvr

/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: mvtnorm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: Matrix

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: stats4

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: gmm

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/fultonw/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: Use

In [3]:
# random seed
np.random.seed(41)

In [4]:
# define params for simulating data
L = 2
N = 1500
d = 2
x_var = 15.0
mu = np.tile(5.0, d)
#prec_val = 100000
prec_val = 1
lambda_val = 1

In [5]:
# simulate the data
x_ns, y_ns, z_ns, B_ls = hvr.simulate_data(L, N, d, x_var, mu, prec_val, lambda_val)

In [6]:
# define prior by specifying hyperparameters
v_0 = 1.0001
assert v_0 > d-1
T_0 = 1.0 * np.eye(d) 
c_0 = .0001 
m_0 = np.zeros(d)
alpha_0 = 1.
beta_0 = 1.

In [7]:
# define params for doing inference
num_iters = 15

In [8]:
# initialize params
v, T, c, m, mu_B_ls, prec_B_ls, alpha_lambda_ls, beta_lambda_ls = hvr.init_variational_params\
(x_ns, y_ns, z_ns, v_0, T_0, c_0, m_0, alpha_0, beta_0)

In [9]:
# do inference
v, T, c, m, mu_B_ls, prec_B_ls, alpha_lambda_ls, beta_lambda_ls = hvr.infer_variational_params(num_iters, x_ns, y_ns, z_ns, v_0, T_0, c_0, m_0, alpha_0, beta_0, \
                         v, T, c, m, mu_B_ls, prec_B_ls, alpha_lambda_ls, beta_lambda_ls)

L: 2 d: 2
0 0.0
3.0001 [[ 0.33333333  0.        ]
 [ 0.          0.33333333]] 2.0001 [ 0.  0.]
temp2 update -1.0 -0.9999500025
1 step 0 mu_B_ls sum: 0.000 log evidence bound: -491355.673825
3.0001 [[ 0.33333333  0.        ]
 [ 0.          0.33333333]] 2.0001 [ 0.  0.]
temp2 update -1.0 -0.9999500025
3.0001 [[ 0.33333333  0.        ]
 [ 0.          0.33333333]] 2.0001 [ 0.  0.]
temp2 update -1.0 -0.9999500025
1 17.9074561413
3.0001 [[ 0.99815924 -0.00712276]
 [-0.00712276  0.90502197]] 2.0001 [ 3.72854761  5.22473279]
temp2 update -1.0 -115.916089735
1 step 1 mu_B_ls sum: 17.907 log evidence bound: -3524.353928
3.0001 [[ 0.99815924 -0.00712276]
 [-0.00712276  0.90502197]] 2.0001 [ 3.72854761  5.22473279]
temp2 update -1.0 -115.916089735
3.0001 [[ 0.99815924 -0.00712276]
 [-0.00712276  0.90502197]] 2.0001 [ 3.72854761  5.22473279]
temp2 update -1.0 -115.916089735
2 17.9090436107
3.0001 [[ 0.99816372 -0.00712153]
 [-0.00712153  0.9050472 ]] 2.0001 [ 3.7288724  5.2252017]
temp2 update -1.0

In [10]:
print 'v',v
print 'T', T
print 'c', c
print 'm', m
print 'mu_B_ls', mu_B_ls
print 'prec_B_ls', prec_B_ls
print 'alpha_lambda_ls', alpha_lambda_ls
print 'beta_lambda_ls', beta_lambda_ls
print y_ns

v 3.0001
T [[ 0.99816373 -0.00712153]
 [-0.00712153  0.90504721]]
c 2.0001
m [ 3.72887249  5.22520181]
mu_B_ls [[ 3.71593641  4.99954725]
 [ 3.74218145  5.45137888]]
prec_B_ls [array([[ 12622.78067922,   -526.34369575],
       [  -526.34369575,  11925.51466217]]), array([[ 12264.00592732,   -343.344495  ],
       [  -343.344495  ,  11577.3107382 ]])]
alpha_lambda_ls [ 376.  376.]
beta_lambda_ls [ 364.56629836  366.41096725]
[ -0.78031392 -17.20991383 -12.30532601 ...,  36.6036112  -17.59062211
   5.27663462]


In [11]:
print np.mean(np.abs(mu_B_ls - B_ls))
print 'estimated', v*T
print 'actual empirical', 1.0/np.var(mu_B_ls[:,0])
print mu_B_ls
print np.concatenate([mu_B_ls,B_ls],axis=1)
print prec_B_ls
print alpha_lambda_ls / beta_lambda_ls

0.00671124342944
estimated [[ 2.99459101 -0.02136531]
 [-0.02136531  2.71523212]]
actual empirical 5807.17958737
[[ 3.71593641  4.99954725]
 [ 3.74218145  5.45137888]]
[[ 3.71593641  4.99954725  3.70913068  5.00360952]
 [ 3.74218145  5.45137888  3.73400098  5.44358238]]
[array([[ 12622.78067922,   -526.34369575],
       [  -526.34369575,  11925.51466217]]), array([[ 12264.00592732,   -343.344495  ],
       [  -343.344495  ,  11577.3107382 ]])]
[ 1.03136248  1.02617016]


In [12]:
h=hvr.trunc_norm()
h.mean(np.array([0,0,0]),np.array([[2,1,1],[1,2,1],[1,1,2]]))

array([ 0.,  0.,  0.])